In [1]:
import pandas as pd 
df = pd.read_excel(r"C:\Users\Shlok\OneDrive\Desktop\financial holdings project.xlsx")
df.head()
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38 entries, 0 to 37
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Symbol                     38 non-null     object        
 1   ISIN                       38 non-null     object        
 2   Sector                     38 non-null     object        
 3   Quantity Available         38 non-null     int64         
 4   Quantity Discrepant        38 non-null     int64         
 5   Quantity Long Term         38 non-null     int64         
 6   Quantity Pledged (Margin)  38 non-null     int64         
 7   Date Buy                   38 non-null     datetime64[ns]
 8   Average Price              38 non-null     float64       
 9   Previous Closing Price     38 non-null     float64       
 10  Unrealized P&L             38 non-null     float64       
 11  Unrealize P&L Pct.         38 non-null     float64       
dtypes: datetim

In [2]:
# Systemize all the column names 
df.columns = (df.columns.str.strip()
.str.lower()
.str.replace("&", "and")
.str.replace("%", "pct")
.str.replace(" ", "_")
.str.replace(".", "")
)

# Check for dublicate 
df.duplicated().sum()

np.int64(0)

In [3]:
# creating columns 

df['Total_invested_valuation'] = df["quantity_available"] * df["average_price"]
df['current_market_valuation'] = df["quantity_available"] * df["previous_closing_price"]
df["Actual_Profit_Loss"] = df['current_market_valuation'] - df['Total_invested_valuation']


In [4]:
df.to_csv("final_stock_holdings.csv", index=False)

In [5]:
# Feature engineering 

# 1 - ROI % (is a measure of how much profit/loss made relative to investment)
df['ROI_pct'] = (df['Actual_Profit_Loss']/ df['Total_invested_valuation']) * 100 

# 2 - Profit/Loss ( we use LAMBDA if any (-ve) value is to vbe found them it will be flagged as a loss )
df['profit_loss_status'] = df['Actual_Profit_Loss'].apply(lambda x: 'Profit' if x > 0 else 'Loss')

# 3 - Find win rate % ( what % of trades in the portfolio is profitable as per data ) 
df['win_rate'] = (df['profit_loss_status'] == 'Profit').mean() * 100

# 4 - Avg profit on profitable trade  (will filter first all the profit and then give us mean)
df['avg_profit'] = df[df['Actual_Profit_Loss'] > 0]['Actual_Profit_Loss'].mean()

# 5 - Avg loss on losing trade  (will filter first all the loosing trade and then give us mean)
df['avg_loss'] = df[df['Actual_Profit_Loss'] < 0]['Actual_Profit_Loss'].mean()

# 6 - Risk–Return Tradeoff(abs will ensure it treats denominator as a number in case of (-ve))
df['risk_reward_ratio'] = abs(df['avg_profit'] / df['avg_loss'])

# 7 - Average holding position 
df['holding_period_days'] = (pd.Timestamp.today() - df['date_buy']).dt.days

# 8 - Total p/l in our portfolio 
df['total_profit_loss'] = df['Actual_Profit_Loss'].sum()



In [35]:
KPI_Overview = {
    "Total_investment": df['Total_invested_valuation'].sum(),
    "profitable_stocks_count": df[df['profit_loss_status'] == 'Profit'].shape[0],
    "Loss_making_stocks_count": df[df['profit_loss_status'] == 'Loss'].shape[0],
    "average_ROI_pct": df['ROI_pct'].mean(),
    "unrealised_profit_loss": df['unrealized_pandl'].sum(),
    "current_market_value_portfolio": df['current_market_valuation'].sum(),
    "most_profitable_stock": df.loc[df['Actual_Profit_Loss'].idxmax(), 'symbol'],
    "least_profitable_stock": df.loc[df['Actual_Profit_Loss'].idxmin(), 'symbol'],
    "top4_profitable_stocks": df.sort_values(by='Actual_Profit_Loss', ascending=False).head(4)['symbol'].tolist(),
    "top3_investment": df.sort_values(by='Total_invested_valuation', ascending=False).head(3)['Total_invested_valuation'].sum(),
    "top3_investment_pct": (df.sort_values(by='Total_invested_valuation', ascending=False).head(3)['Total_invested_valuation'].sum() / df['Total_invested_valuation'].sum()) * 100,
    "longterm_pct": (df['quantity_long_term'].sum() / df['quantity_available'].sum()) * 100,
    "pledged_pct": (df['quantity_pledged_(margin)'].sum() / df['quantity_available'].sum()) * 100,
    "best_sector": (df.sort_values(by='Actual_Profit_Loss', ascending=False).groupby('sector').first().reset_index()[['sector', 'symbol', 'Actual_Profit_Loss', 'Total_invested_valuation']])
}


In [37]:
KPI_df = pd.DataFrame(KPI_Overview.items(), columns=["KPI", "Value"])
KPI_df.to_csv("portfolio_KPIs.csv", index=False)
